# Fine-Tuning DistilBERT
The goal is to Fine-tune DistilBERT to predict sentiment on the Twitter dataset.

## About Dataset
#### Context
This is the sentiment140 dataset. It contains 1,600,000 tweets extracted using the twitter api . The tweets have been annotated (0 = negative, 4 = positive) and they can be used to detect sentiment .

#### Content
It contains the following 6 fields:

target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

ids: The id of the tweet ( 2087)

date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)

flag: The query (lyx). If there is no query, then this value is NO_QUERY.

user: the user that tweeted (robotickilldozr)

text: the text of the tweet (Lyx is cool)

#### Acknowledgements
The official link regarding the dataset with resources about how it was generated is here
The official paper detailing the approach is here

#### Citation: 
Go, A., Bhayani, R. and Huang, L., 2009. Twitter sentiment classification using distant supervision. CS224N Project Report, Stanford, 1(2009), p.12.



In [13]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)
os.getcwd()


'C:\\Users\\Alex Chung\\Documents\\the_Lab\\Portfolio\\ml_engineering\\notebooks'

## 1. Loading Data

In [2]:
path = "c:\\Users\\Alex Chung\\Documents\\the_Lab\\Portfolio\\ml_engineering\\data\\sentiment140\\"
df = pd.read_csv(path+"training.1600000.processed.noemoticon.csv", encoding="ISO-8859-1", names=["target", "id", "date", "flag", "user", "text"])

In [5]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id      1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [18]:
df.target.value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [10]:
df.text.to_list()[:2]

["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D",
 "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"]

## 2. Preprocessing

target
0    800000
4    800000
Name: count, dtype: int64

In [20]:
# Split the data into train and test set
train, test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

print(f"New stratefied dataframe shapes: train is {train.shape}, train is {test.shape}")



New stratefied dataframe shapes: train is (1280000, 6), train is (320000, 6)


In [22]:
print("Train target counts:")
train.target.value_counts()

Train counts:


target
4    640000
0    640000
Name: count, dtype: int64

In [23]:
print("Test target counts:")
test.target.value_counts()

Test counts:


target
0    160000
4    160000
Name: count, dtype: int64

In [11]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model = )
classifier(df.text.to_list()[:2])

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

C:\Users\Alex Chung\Documents\the_Lab\Portfolio\ml_engineering\venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Alex Chung\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo,

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

NameError: name 'init_empty_weights' is not defined

## 3. Model Training

## 4. Evaluation

## 1. Loading Data